In [1]:
from transformers import pipeline
import pandas as pd
import torch
from tqdm import tqdm

2023-04-19 22:51:44.161224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 22:51:45.932499: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-19 22:51:45.932571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-19 22:51:45.932579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
data = pd.read_csv("/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/war_data/training_data/secondary.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54043 entries, 0 to 54042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               54043 non-null  object
 1   comment_text     54043 non-null  object
 2   toxicity         54043 non-null  int64 
 3   severe_toxicity  54043 non-null  int64 
 4   obscene          54043 non-null  int64 
 5   threat           54043 non-null  int64 
 6   insult           54043 non-null  int64 
 7   identity_attack  54043 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 3.3+ MB


In [3]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [4]:
candidate_labels = [
                    'USA started the war between Russia and Ukraine',
                    'POTUS started the war between Russia and Ukraine',
                    'Joe Biden started the war between Russia and Ukraine',
                    'CIA started the war between Russia and Ukraine',
                    'USA influenced the war between Russia and Ukraine',
                    'POTUS influenced the war between Russia and Ukraine',
                    'Joe Biden influenced the war between Russia and Ukraine',
                    'CIA influenced the war between Russia and Ukraine',
                    ]

In [6]:
print("Starting analysis")
tweets_blaming_america = {}
for tweet in tqdm(data.sample(250)['comment_text']):
    result = classifier(tweet, candidate_labels, multi_label=True)
    if all(val < 0.2 for val in result['scores']):
        tweets_blaming_america[tweet] = {l: s for l, s in zip(result['labels'], result['scores'])}

Starting analysis


100%|██████████| 250/250 [05:20<00:00,  1.28s/it]


In [8]:
print(len(tweets_blaming_america))

43


In [7]:
for tweet, result in tweets_blaming_america.items():
    print(tweet.replace("\n", " "))
    for label, score in result.items():
        print(f"\t{label}: {score}")

POTUS owes Ukraine for the millions they gave his son and the REAL collusion he had to get a prosecutor fired, and American troops are the payment. What a crook.
	USA influenced the war between Russia and Ukraine: 0.13656117022037506
	POTUS influenced the war between Russia and Ukraine: 0.021981684491038322
	CIA influenced the war between Russia and Ukraine: 0.007433576975017786
	Joe Biden influenced the war between Russia and Ukraine: 0.0011769578559324145
	USA started the war between Russia and Ukraine: 0.0009756264626048505
	CIA started the war between Russia and Ukraine: 0.0004285851900931448
	POTUS started the war between Russia and Ukraine: 0.00018153461860492826
	Joe Biden started the war between Russia and Ukraine: 8.165321196429431e-05
Get Ready for US troops in the Ukraine? From Afghanistan to the Ukraine Let's go Brandon
	USA influenced the war between Russia and Ukraine: 0.07310007512569427
	POTUS influenced the war between Russia and Ukraine: 0.03064044751226902
	CIA influ

In [9]:
tests = ['Lying propagandist media shouting invasion over Ukraine, yet remain silent on the ACTUAL INVASION of America at our southern border']
for tweet in tests:
    result = classifier(tweet, candidate_labels, multi_label=True)
    print(tweet)
    for label, score in zip(result['labels'], result['scores']):
        print(f"\t{label}: {score}")

Lying propagandist media shouting invasion over Ukraine, yet remain silent on the ACTUAL INVASION of America at our southern border
	USA influenced the war between Russia and Ukraine: 0.0728783905506134
	CIA influenced the war between Russia and Ukraine: 0.00525757297873497
	POTUS influenced the war between Russia and Ukraine: 0.0025946798268705606
	USA started the war between Russia and Ukraine: 0.0016453873831778765
	Joe Biden influenced the war between Russia and Ukraine: 0.0016119081992655993
	CIA started the war between Russia and Ukraine: 0.00143335887696594
	POTUS started the war between Russia and Ukraine: 0.0006974498392082751
	Joe Biden started the war between Russia and Ukraine: 0.0006885910406708717
